In [2]:
import numpy as np
from collections import deque
mat = np.array([[0, 1, 1, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 1, 1, 0]])

expected_dist = np.array([[0,1,1,2,3,3,3],
                          [1,0,1,1,2,2,2],
                          [1,1,0,2,3,3,3],
                          [2,1,2,0,1,1,1],
                          [3,2,3,1,0,2,1],
                          [3,2,3,1,2,0,1],
                          [3,2,3,1,1,1,0]])

In [3]:
num_vertices = mat.shape[0]
res = np.full((num_vertices, num_vertices), np.inf)

# Finish this loop
for i in range(num_vertices):
    Q = deque()
    # initialize visited vector
    visited = np.full((num_vertices), False)
    
    # set diagonal elements to distance of 0
    np.fill_diagonal(res,0)
    
    # set first vertex in queue and set as visited
    Q.append([i, 0])
    visited[i] = True

    # run through network while there are still vertices left in the queue
    while Q:
        # current reference distance
        distance = Q[0][1]

        # determine adjacent vertices
        adj_vert = np.where(mat[Q[0][0]] > 0)[0]
        
        # run if there exists any adjacent vertices
        if adj_vert.any():
            
            # filter out vertices that have already been visited
            adj_vert = [vert for vert in adj_vert if visited[vert] == False]
            
            # add adjacent vertices to queue and set them as visited
            [Q.append([vert,distance + 1]) for vert in adj_vert]
            np.put(visited, adj_vert, True)

            # record the distance to the vertices
            np.put(res[i], adj_vert,distance+1)
            
            # remove current vertex from queue
            Q.popleft()

print(res)

[[0. 1. 1. 2. 3. 3. 3.]
 [1. 0. 1. 1. 2. 2. 2.]
 [1. 1. 0. 2. 3. 3. 3.]
 [2. 1. 2. 0. 1. 1. 1.]
 [3. 2. 3. 1. 0. 2. 1.]
 [3. 2. 3. 1. 2. 0. 1.]
 [3. 2. 3. 1. 1. 1. 0.]]


In [4]:
dist_mat = res
num_vertices = mat.shape[0]
available = [True for _ in range(num_vertices)]
expected_components = [np.array([0,1,2]), np.array([3,4,5,6])]

components = []

# finish this loop
while any(available):
    current_node = available.index(True)

    components.append(np.where(dist_mat[current_node] < np.inf)[0].tolist())
    available[current_node] = False

components = np.unique(components)
print(components)
print(expected_components)

[0 1 2 3 4 5 6]
[array([0, 1, 2]), array([3, 4, 5, 6])]


In [5]:
short_mat = np.array([[0, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 0, 1, 2],
       [1, 1, 1, 1, 2, 0 ,1],
       [1, 1, 1, 1, 1, 1, 0]])

# count of how many shortest parths use that edge for each starting vertex
# there are 7 vertices in this matrix. so matrix is 7x7, but not all vertex pairs will have an
# edge, so those will be 0
count = np.array([[0, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 0, 1, 2],
       [1, 1, 1, 1, 2, 0 ,1],
       [1, 1, 1, 1, 1, 1, 0]])

In [6]:
num_vertices = mat.shape[0]
    paths_matrix = np.full(num_vertices, np.inf)
    edge_count = np.full((num_vertices, num_vertices), 0.0)

    Q = deque()
    leaf = []

    # initialize visited and layer vectors
    visited = np.full((num_vertices), False)
    layer = np.full((num_vertices), np.inf)

    # set first vertex in queue and set as visited
    Q.append([vertex, 1])
    visited[vertex] = True
    layer[vertex] = 1
    paths_matrix[vertex] = 1

    # run through network while there are still vertices left in the queue
    while Q:
        # current reference number of shortest paths
        path = Q[0][1]

        # determine adjacent vertices
        adj_vert = np.where(mat[Q[0][0]] > 0)[0]

        # filter out parent and sibling vertices
        adj_vert = [vert for vert in adj_vert if layer[vert] > layer[Q[0][0]]]

        # run if there exists any viable adjacent vertices
        if adj_vert:

            # record the number of shortest paths to the vertices and their layer
            for j in adj_vert:
                if paths_matrix[j] == np.inf:
                    np.put(paths_matrix, j, path)
                else:
                    np.put(paths_matrix, j, paths_matrix[j] + path)

            for k in adj_vert:
                layer[k] = layer[Q[0][0]] + 1

            # add adjacent vertices to queue and set them as visited
            [Q.append([vert, paths_matrix[vert]]) for vert in adj_vert]
            np.put(visited, adj_vert, True)

            # remove current vertex from queue
            Q.popleft()

        # if there is no viable adjacent vertex, set this vertex as a leaf
        else:
            leaf.append(Q[0][0])
            Q.popleft()

    ## calculate edge counts
    layer_copy = layer
    layer_copy = sorted(set(layer_copy), reverse=True)
    layer_copy.remove(layer_copy[0])

    # assign credit of 1 to each leaf
    credit = np.full(num_vertices, 1.0)
    # np.put(credit,leaf,1)

    # at each layer, except bottom-most one
    for j in layer_copy:

        # for every vertex that is in this layer
        for v, l in np.ndenumerate(layer):
            if l == j:

                # for every vertex in lower layer that is adjacent to this vertex
                for v_lower, l_lower in np.ndenumerate(layer):
                    if l_lower == l + 1 and mat[v][v_lower] == 1:
                        # divide shortest path count of current layer vertex (v) by that of the lower layer vertex (v_lower) multiply by the credit of the lower vertex.

                        # print('v=',v[0],'v_lower=',v_lower[0])
                        # print('v credit=',credit[v[0]],', v_lower credit=',credit[v_lower[0]])
                        # print('v path = ',paths_matrix[v], "v_lower path = ",paths_matrix[v_lower])

                        edge_credit = (
                            paths_matrix[v] / paths_matrix[v_lower]
                        ) * credit[v_lower[0]]

                        # add this edge credit the the appropriate place in edge_count matrix at both locations
                        edge_count[v][v_lower] = edge_credit
                        edge_count[v_lower][v] = edge_credit

                        # update credit recieved from each child vertex
                        credit[v[0]] += edge_count[v][v_lower]

                        # print("edge_count = ",edge_count[v][v_lower])
                        # print('----------------v credit =',credit[v[0]],"\n")

IndentationError: unexpected indent (<ipython-input-6-05fb773166cd>, line 2)

In [47]:
from networks_lib.communities import *
from networks_lib.distance import *

K = 2

num_vertices = mat.shape[0]

# make a copy of matrix since we are going
# to update it as we remove edges
work_mat = mat.copy()

components = get_components(mat)

# if only 1 component, get_components() returns a 1D list with all vertices....convert this to 2D list for while loop to work
if isinstance(components[0],list):
    pass
else:
    components = [components]
    
while len(components) < K:
    #print(components)
    # compute edge betweenness (one component at a time)
    eb = np.zeros((num_vertices, num_vertices))
    for vertices in components:
        cur_mat = work_mat[vertices,:][:, vertices]
        cur_eb = edge_betweenness(cur_mat)
        for i in range(len(vertices)):
            eb[vertices[i], vertices] = cur_eb[i, :]
    
    # find location of maximum betweeness
    edge = np.where(eb == eb.max())

    # remove that edge
    work_mat[edge[0][0],edge[0][1]] = 0
    work_mat[edge[0][1],edge[0][0]] = 0

    # get new components
    components = get_components(work_mat)

    if isinstance(components[0],list):
        pass
    else:
        components = [components]
    #print(components)
        
#print(components_to_assignment(components, num_vertices))

[array([0, 1, 2, 3, 4, 5, 6])]
[list([0, 1, 2]) list([3, 4, 5, 6])]


In [42]:
sol = np.where(eb == eb.max())
print(mat)
print(sol)

for i in sol:
    print(mat[i[0],i[1]])

[[0 1 1 0 0 0 0]
 [1 0 1 1 0 0 0]
 [1 1 0 0 0 0 0]
 [0 1 0 0 1 1 1]
 [0 0 0 1 0 0 1]
 [0 0 0 1 0 0 1]
 [0 0 0 1 1 1 0]]
(array([1, 3]), array([3, 1]))
1
1


In [ ]:
type(components[0])